# 사용 패키지

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

# 데이터 로드

In [2]:
total = pd.read_csv('./data/한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')
test = pd.read_csv("./data/test.csv")

# 전처리

In [3]:
# 구분(공급사)를 A~H를 0~6 까지 맵핑시킴 
d_map = {} 
for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map)
total['연월일'] = pd.to_datetime(total['연월일'])
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday
total['month_day'] = (total['month']*100) + total['day']
total['year_month_day'] = (total['year']*10000)+(total['month']*100) + total['day']

total.columns= ['date','time','s_chain','s_qty','year','month','day','weekday','month_day','year_month_day']
#total

In [4]:
test['date'] = test['일자|시간|구분'].str.split(' ').str[0]
test['time'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['s_chain'] = test['일자|시간|구분'].str.split(' ').str[2]
test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['weekday'] = test['date'].dt.weekday
test['month_day'] = (test['month']*100) + test['day']

d_map = {} 
for i, d in enumerate(test['s_chain'].unique()):
    d_map[d] = i
test['s_chain'] = test['s_chain'].map(d_map)

test.columns= ['일시구','date','time','s_chain','year','month','day','weekday','month_day']
#test

### 전체에서 이상치 제외하고 평균을 낸 후 대체하는 코드

In [5]:
h_thres = 7000
l_thres = 10

strange_total = total[(total.s_qty <= l_thres)|(total.s_qty >= h_thres)]
#strange_total

In [6]:
lower_idx = total[total.s_qty <= l_thres].index
upper_idx = total[total.s_qty >= h_thres].index
del_idx = lower_idx.append(upper_idx)
total_ex_strange = total.drop(index=(del_idx))
#total_ex_strange

In [7]:
total_group_by = total_ex_strange.groupby(['month','day','time','s_chain']).mean()
#total_group_by

In [8]:
for i in strange_total.index:
    total.loc[i,'s_qty'] = total_group_by.loc[(strange_total.loc[i,'month'], strange_total.loc[i,'day'], strange_total.loc[i,'time'], strange_total.loc[i,'s_chain']), 's_qty']
    

### train, test 파일 공휴일 추가

In [9]:
holy = pd.read_csv('./make_data/0A_holiday.csv')

holy['holi'] = holy['holi'].astype('int')
holy.loc[(holy['dayname']==1)|(holy['holi']==1), 'no_working'] = 1
holy['no_working'].fillna(0, inplace=True)

holy2 = holy[['date', 'no_working']]
#holy2

In [10]:
total['date'] = total['date'].astype(str)
train_df = pd.merge(total, holy2, how='left', on='date')
#gas_df

In [11]:
test['date'] = test['date'].astype('str')
test_df = pd.merge(test, holy2, how='left', on='date') #no_working 데이터 merge

### train test 파일 음력 월,일 추가 

In [12]:
lun_df = pd.read_csv("./make_data/0B_음력날짜.csv")
lun_df2 = lun_df[['lunMonth','lunDay','date']]
train_df2 = pd.merge(train_df, lun_df2, how='left', on='date')
test_df2 = pd.merge(test_df, lun_df2, how='left', on='date')

### train 파일 관측 온도 추가

In [13]:
train_temp = pd.read_csv("./make_data/0D_train_temp_서울.csv") 

In [14]:
train_temp

,date,temp
0,2013-01-01 00:00,-8.3
1,2013-01-01 01:00,-8.5
2,2013-01-01 02:00,-8.4
3,2013-01-01 03:00,-8.1
4,2013-01-01 04:00,-8.2
...,...,...
52579,2018-12-31 19:00,-3.1
52580,2018-12-31 20:00,-3.7
52581,2018-12-31 21:00,-4.6
52582,2018-12-31 22:00,-5.4


In [15]:
train_temp['time']= train_temp['date'].str[11:13].astype(int)+1
train_temp['time'] = train_temp['time'].astype(int)
train_temp['date']= train_temp['date'].str[:10]
#train_temp

In [16]:
train_df3 = pd.merge(train_df2, train_temp, how = 'left', on=['date','time'])
train_df3

,date,time,s_chain,s_qty,year,month,day,weekday,month_day,year_month_day,no_working,lunMonth,lunDay,temp
0,2013-01-01,1,0,2497.129,2013,1,1,1,101,20130101,1.0,11,20,-8.3
1,2013-01-01,2,0,2363.265,2013,1,1,1,101,20130101,1.0,11,20,-8.5
2,2013-01-01,3,0,2258.505,2013,1,1,1,101,20130101,1.0,11,20,-8.4
3,2013-01-01,4,0,2243.969,2013,1,1,1,101,20130101,1.0,11,20,-8.1
4,2013-01-01,5,0,2344.105,2013,1,1,1,101,20130101,1.0,11,20,-8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,1231,20181231,0.0,11,25,-3.1
368084,2018-12-31,21,6,669.961,2018,12,31,0,1231,20181231,0.0,11,25,-3.7
368085,2018-12-31,22,6,657.941,2018,12,31,0,1231,20181231,0.0,11,25,-4.6
368086,2018-12-31,23,6,610.953,2018,12,31,0,1231,20181231,0.0,11,25,-5.4


### 절기 추가
###### 입춘 음력  2월  3일
###### 입하 음력  3월 24일
###### 입추 음력  6월 29일
###### 입동 음력 11월  7일

In [17]:
train_df3['lun_month_day'] = (train_df3['lunMonth']*100) + train_df3['lunDay']
test_df2['lun_month_day'] = (test_df2['lunMonth']*100) + test_df2['lunDay']

In [18]:
def get_season(x):
    if x < 203:
        return 1
    elif (x >= 203)&(x < 324):
        return 2
    elif (x >= 324)&(x < 629):
        return 3
    elif (x >= 629)&(x < 1107):
        return 4
    else:
        return 1

train_df3['season'] = train_df3['lun_month_day'].apply(get_season)
test_df2['season'] = test_df2['lun_month_day'].apply(get_season)

### train, test 파일 일출 일몰 추가

In [19]:
train_sun = pd.read_csv("./make_data/0C_train_일출일몰_예보.csv")
test_sun = pd.read_csv("./make_data/0C_test_일출일몰_예보.csv") #test 일사는 평균을 구해서 넣음

train_sun = train_sun[['date','sunrise','sunset','일사']]

In [20]:
train_df4 = pd.merge(train_df3, train_sun, how='left', on='date')
test_df3 = pd.merge(test_df2, test_sun, how='left', on='date')

In [21]:
#### 공급사 별로 슬라이싱해서 하는 부분 1~3월만

# 1~3월만 뽑기
gas_1to3 = train_df4[(train_df4.month==1)|(train_df4.month==2)|(train_df4.month==3)]

#공급사별로 슬라이싱
for i in gas_1to3['s_chain'].unique() :
    globals()['df_{}'.format(i)] = gas_1to3[gas_1to3['s_chain']==i]

In [22]:
#평균으로 대체하는 형식에 필요한 함수들

def thres(df,h_thres,l_thres): #이상치를 뽑음
    strange = pd.DataFrame()
    strange = df[(df.s_qty <= l_thres)|(df.s_qty >= h_thres)]
    return strange



def index_del(df,h_thres,l_thres): #이상치 제외한걸 뽑음
    lower_idx = df[df.s_qty <= l_thres].index
    upper_idx = df[df.s_qty >= h_thres].index
    del_idx = lower_idx.append(upper_idx)
    ex_strange = df.drop(index=(del_idx))
    
    return ex_strange   



def replace_strange(df_strange,df_group_by,total):
    for i in df_strange.index:
        total.loc[i,'s_qty'] = df_group_by.loc[(df_strange.loc[i,'month'], df_strange.loc[i,'day'], df_strange.loc[i,'time'], df_strange.loc[i,'s_chain']), 's_qty']
    

In [23]:
df1_strange = thres(df_1,4000,700)
df1_ex_strange = index_del(df_1,4000,700)
df1_group_by = df1_ex_strange.groupby(['month','day','time','s_chain']).mean()
replace_strange(df1_strange,df1_group_by,train_df4)

In [24]:
del_idx = train_df4[(train_df4.year_month_day>=20180123)&(train_df4.year_month_day<20180214)&(train_df4.s_chain==2)].index

train_df4.drop(index=del_idx, inplace=True )

In [25]:
df5_strange = thres(df_5,7000,1500)
df5_ex_strange = index_del(df_5,7000,1500)
df5_group_by = df5_ex_strange.groupby(['month','day','time','s_chain']).mean()
replace_strange(df5_strange,df5_group_by,train_df4)

In [26]:
#2013
chain6 = train_df4[(train_df4.year==2013)&(train_df4.s_chain==6)]
chain6 = chain6[(chain6.month==10)] 
chain6 = chain6[chain6.s_qty > 600]

In [27]:
idx = chain6.index
chain6_ex_strange = train_df4.drop(index=(idx))

In [28]:
chain6_group_by = chain6_ex_strange.groupby(['month','day','time','s_chain']).mean()

In [29]:
for i in chain6.index:
    train_df4.loc[i,'s_qty'] = chain6_group_by.loc[(chain6.loc[i,'month'], chain6.loc[i,'day'], chain6.loc[i,'time'], chain6.loc[i,'s_chain']), 's_qty']
    

In [30]:
#2015
chain615 = train_df4[(train_df4.year==2015)&(train_df4.s_chain==6)]
chain615 = chain615[(chain615.month==6)|(chain615.month==7)] 
chain615 = chain615[chain615.s_qty > 200]

In [31]:
idx = chain615.index
chain615_ex_strange = train_df4.drop(index=(idx))

In [32]:
chain615_group_by = chain615_ex_strange.groupby(['month','day','time','s_chain']).mean()

In [33]:
for i in chain615.index:
    train_df4.loc[i,'s_qty'] = chain615_group_by.loc[(chain615.loc[i,'month'], chain615.loc[i,'day'], chain615.loc[i,'time'], chain615.loc[i,'s_chain']), 's_qty']
    

In [34]:
#2016 4월
chain6_16_4 = train_df4[(train_df4.year==2016)&(train_df4.s_chain==6)] 
chain6_16_4 = chain6_16_4[chain6_16_4.s_qty > 850]

In [35]:
idx = chain6_16_4.index
chain6_16_4_ex_strange = train_df4.drop(index=(idx))

In [36]:
chain6_16_4_group_by = chain6_16_4_ex_strange.groupby(['month','day','time','s_chain']).mean()

In [37]:
for i in chain6_16_4.index:
    train_df4.loc[i,'s_qty'] = chain6_16_4_group_by.loc[(chain6_16_4.loc[i,'month'], chain6_16_4.loc[i,'day'], chain6_16_4.loc[i,'time'], chain6_16_4.loc[i,'s_chain']), 's_qty']
    

In [38]:
#2016,2017
chain66 = train_df4[(train_df4.year==2016)|(train_df4.year==2017)] 
chain66 = chain66[(chain66.s_chain==6)]
chain66 = chain66[chain66.s_qty > 1000]

In [39]:
idx = chain66.index
chain66_ex_strange = train_df4.drop(index=(idx))

In [40]:
chain66_group_by = chain66_ex_strange.groupby(['month','day','time','s_chain']).mean()

In [41]:
for i in chain66.index:
    train_df4.loc[i,'s_qty'] = chain66_group_by.loc[(chain66.loc[i,'month'], chain66.loc[i,'day'], chain66.loc[i,'time'], chain66.loc[i,'s_chain']), 's_qty']
    

In [42]:
train_df4

,date,time,s_chain,s_qty,year,month,day,weekday,month_day,year_month_day,no_working,lunMonth,lunDay,temp,lun_month_day,season,sunrise,sunset,일사
0,2013-01-01,1,0,2497.129,2013,1,1,1,101,20130101,1.0,11,20,-8.3,1120,1,7,17,10
1,2013-01-01,2,0,2363.265,2013,1,1,1,101,20130101,1.0,11,20,-8.5,1120,1,7,17,10
2,2013-01-01,3,0,2258.505,2013,1,1,1,101,20130101,1.0,11,20,-8.4,1120,1,7,17,10
3,2013-01-01,4,0,2243.969,2013,1,1,1,101,20130101,1.0,11,20,-8.1,1120,1,7,17,10
4,2013-01-01,5,0,2344.105,2013,1,1,1,101,20130101,1.0,11,20,-8.2,1120,1,7,17,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,1231,20181231,0.0,11,25,-3.1,1125,1,7,17,10
368084,2018-12-31,21,6,669.961,2018,12,31,0,1231,20181231,0.0,11,25,-3.7,1125,1,7,17,10
368085,2018-12-31,22,6,657.941,2018,12,31,0,1231,20181231,0.0,11,25,-4.6,1125,1,7,17,10
368086,2018-12-31,23,6,610.953,2018,12,31,0,1231,20181231,0.0,11,25,-5.4,1125,1,7,17,10


In [43]:
train_df4.to_csv("./make_data/1_final_train.csv",index=False)
test_df3.to_csv("./make_data/1_final_test.csv",index=False)

# End 
============================================================================================